In [3]:
class ValidationPolygon(type):
    def __new__(meta, name, bases, class_dict):
        #Don't validate the abstract Polygon class
        if bases != (object, ):
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)
    
class Polygon(object):
    __metaclass__ = ValidationPolygon
    
    sides = None
    
    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180
    
class Triangle(Polygon):
    sides = 3

In [5]:
print 'Before class'
class Line(Polygon):
    print 'Before sides'
    sides = 1
    print 'After sides'
    
print 'After class'

Before class
Before sides
After sides


ValueError: Polygons need 3+ sides

In [9]:
import json

class BetterSerializable(object):
    def __init__(self, *args):
        self.args = args
        
    def serialize(self):
        return json.dumps({
                'class': self.__class__.__name__,
                'args': self.args,
            })
    
registry = {}

def register_class(target_class):
    registry[target_class.__name__] = target_class
    
def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])

class Meta(type):
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)
        return cls
    
class RegisteredSerializable(BetterSerializable):
    __metaclass__ = Meta
    
class Vector3D(RegisteredSerializable):
    def __init__(self, x, y, z):
        super(Vector3D, self).__init__(x, y, z)
        self.x, self.y, self.z = x, y, z
    
    def __repr__(self):
        return "Vector3D(%d, %d, %d)" % (self.x, self.y, self.z)
    
v3 = Vector3D(10, -7, 3)
print 'Before:', v3
data = v3.serialize()
print 'Serialized:', data
print 'After:', deserialize(data)

Before: Vector3D(10, -7, 3)
Serialized: {"args": [10, -7, 3], "class": "Vector3D"}
After: Vector3D(10, -7, 3)


In [10]:
class Field(object):
    def __init__(self):
        # these will be assigned by the metaclass.
        self.name = None
        self.internal_name = None
        
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)
        
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        
        cls = type.__new__(meta, name, bases, class_dict)
        return cls
    
class DatabaseRow(object):
    __metaclass__ = Meta
    
class BetterCustomer(DatabaseRow):
    first_name = Field()
    last_name = Field()
    
foo = BetterCustomer()
print 'Before:', repr(foo.first_name), foo.__dict__
foo.first_name = 'Euler'
print 'After:', repr(foo.first_name), foo.__dict__

Before: '' {}
After: 'Euler' {'_first_name': 'Euler'}
